<a href="https://colab.research.google.com/github/Dionysus326/Helper_Tools/blob/master/seresnext_v1_aug_128scale1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.1

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.1`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install albumentations
# !pip install git+https://github.com/qubvel/efficientnet
!pip install git+https://github.com/titu1994/keras-squeeze-excite-network

     |████████████████████████████████| 634kB 2.8MB/s 
  Created wheel for imgaug: filename=imgaug-0.2.6-cp36-none-any.whl size=654020 sha256=53b9d926e4e1cac448fb0b2567101135b95223af48e93d87c226c723be2c3f0a
  Stored in directory: /root/.cache/pip/wheels/97/ec/48/0d25896c417b715af6236dbcef8f0bed136a1a5e52972fc6d0
Successfully built imgaug
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully uninstalled imgaug-0.2.9
  Cloning https://github.com/titu1994/keras-squeeze-excite-network to /tmp/pip-req-build-lk09r6l2
  Running command git clone -q https://github.com/titu1994/keras-squeeze-excite-network /tmp/pip-req-build-lk09r6l2
  Created wheel for keras-squeeze-excite-network: filename=keras_squeeze_excite_network-0.0.4-cp36-none-any.whl size=29433 sha256=c0fce7f71eb58940139acbb2435f9166e09b07335d2f2d5f10da0499ce161701
  Stored in directory: /tmp/pip-ephem-wheel-cache-g7xr_u3m/wheels/6a/ba/01/b3346e82b4bec9416759fde654723d42955c88bb0f27cf9689
Succes

In [4]:
!cp /content/drive/My\ Drive/DataSciProject/Bangli/data/Bang_128.tar.gz /content/
!tar -zxvf Bang_128.tar.gz

Streaming output truncated to the last 5000 lines.
resized_train/Train_114769.png
resized_train/Train_34105.png
resized_train/Train_54861.png
resized_train/Train_144065.png
resized_train/Train_48382.png
resized_train/Train_189806.png
resized_train/Train_10233.png
resized_train/Train_163256.png
resized_train/Train_57391.png
resized_train/Train_49672.png
resized_train/Train_81741.png
resized_train/Train_156190.png
resized_train/Train_190371.png
resized_train/Train_3180.png
resized_train/Train_99239.png
resized_train/Train_129762.png
resized_train/Train_184516.png
resized_train/Train_148181.png
resized_train/Train_2332.png
resized_train/Train_151650.png
resized_train/Train_21926.png
resized_train/Train_180.png
resized_train/Train_817.png
resized_train/Train_143537.png
resized_train/Train_7752.png
resized_train/Train_99672.png
resized_train/Train_91306.png
resized_train/Train_33042.png
resized_train/Train_173031.png
resized_train/Train_177159.png
resized_train/Train_107414.png
resized_trai

In [0]:
!mkdir cache
!cp /content/drive/My\ Drive/DataSciProject/Bangli/data/train.csv /content/
!rm Bang_128.tar.gz
!rm -r resized_test
!rm -r sample_data

In [0]:
import pandas as pd
import numpy as np
import os
import glob
import gc
import shutil
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
# import efficientnet.tfkeras as efn 
from keras_squeeze_excite_network import se_resnext
from sklearn.model_selection import KFold
import cv2
AUTOTUNE = tf.data.experimental.AUTOTUNE

gm_exp = tf.Variable(3.0, dtype = tf.float32)
# total 200840
SCALE = 1
HEIGHT = int(128*SCALE)
WIDTH = int(128*SCALE)
BATCH_SIZE = 8
CHANNELS = 3
VAL_RATIO = 0.2
EPOCHS = 60
DEBUG = False
model = None
cache_reset = True

IMG_AUG_PROB = 0.15
val_part = 2

train_csv_path = "/content/train.csv"
train_img_path = "/content/resized_train"
cache_path = "/content/cache/tf_cache_"
model_histories_path = "/content/drive/My Drive/DataSciProject/Bangli/model/0314_seresnext_v1_aug_128scale1.0_histories.csv"
model_path = "/content/drive/My Drive/DataSciProject/Bangli/model/0314_seresnext_v1_aug_128scale1.0_{epoch:02d}+20_{val_loss:.2f}.h5"

if glob.glob(cache_path+"*"):
    if cache_reset:
        clean = "y"
    else:
        clean = input(f"Clean the cache file in {cache_path}??(y/n)")   
    if cache_reset:
        clean = "y"
    if clean == "y":
        shutil.rmtree('cache')
        os.mkdir("cache")
    elif clean == "n":
        pass
    else:
        raise ValueError("enter y or n")

    

In [7]:
label = pd.read_csv(train_csv_path).drop(["grapheme"], axis=1)

if DEBUG:
    label = label.iloc[:int(label.shape[0]*0.05)]

label["image_id"] = label["image_id"].apply(lambda x: train_img_path + f"/{x}.png")
img_path = label.pop("image_id")

kf = KFold(n_splits=5, shuffle=True, random_state=17)
for i, (train_index, val_index) in enumerate(kf.split(img_path)):
    if i == (val_part-1):
        break


ds_x_train = tf.data.Dataset.from_tensor_slices((img_path[train_index]))
ds_y_train = tf.data.Dataset.from_tensor_slices((
    tf.one_hot(label.iloc[train_index]["grapheme_root"], 168),
    tf.one_hot(label.iloc[train_index]["vowel_diacritic"], 11),
    tf.one_hot(label.iloc[train_index]["consonant_diacritic"], 7)
    ))
ds_train = tf.data.Dataset.zip((ds_x_train, ds_y_train))

ds_x_val = tf.data.Dataset.from_tensor_slices((img_path[val_index]))
ds_y_val = tf.data.Dataset.from_tensor_slices((
    tf.one_hot(label.iloc[val_index]["grapheme_root"], 168),
    tf.one_hot(label.iloc[val_index]["vowel_diacritic"], 11),
    tf.one_hot(label.iloc[val_index]["consonant_diacritic"], 7)
    ))
ds_val = tf.data.Dataset.zip((ds_x_val, ds_y_val))

if len(train_index)%BATCH_SIZE != 0:
    print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    print(f"{len(train_index)%BATCH_SIZE} train data will be dropped!")


del label
del img_path
del train_index
del val_index
del kf

# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Base augmentations operators."""

import numpy as np
from PIL import Image, ImageOps, ImageEnhance
from albumentations.core.transforms_interface import ImageOnlyTransform


def int_parameter(level, maxval):
  """Helper function to scale `val` between 0 and maxval .
  Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
  Returns:
    An int that results from scaling `maxval` according to `level`.
  """
  return int(level * maxval / 10)


def float_parameter(level, maxval):
  """Helper function to scale `val` between 0 and maxval.
  Args:
    level: Level of the operation that will be between [0, `PARAMETER_MAX`].
    maxval: Maximum value that the operation can have. This will be scaled to
      level/PARAMETER_MAX.
  Returns:
    A float that results from scaling `maxval` according to `level`.
  """
  return float(level) * maxval / 10.


def sample_level(n):
  return np.random.uniform(low=0.1, high=n)


def autocontrast(pil_img, _):
  return ImageOps.autocontrast(pil_img)


def equalize(pil_img, _):
  return ImageOps.equalize(pil_img)


def posterize(pil_img, level):
  level = int_parameter(sample_level(level), 4)
  return ImageOps.posterize(pil_img, 4 - level)


def rotate(pil_img, level):
  degrees = int_parameter(sample_level(level), 30)
  if np.random.uniform() > 0.5:
    degrees = -degrees
  return pil_img.rotate(degrees, resample=Image.BILINEAR)


def solarize(pil_img, level):
  level = int_parameter(sample_level(level), 256)
  return ImageOps.solarize(pil_img, 256 - level)


def shear_x(pil_img, level):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((WIDTH, HEIGHT),
                           Image.AFFINE, (1, level, 0, 0, 1, 0),
                           resample=Image.BILINEAR)


def shear_y(pil_img, level):
  level = float_parameter(sample_level(level), 0.3)
  if np.random.uniform() > 0.5:
    level = -level
  return pil_img.transform((WIDTH, HEIGHT),
                           Image.AFFINE, (1, 0, 0, level, 1, 0),
                           resample=Image.BILINEAR)


def translate_x(pil_img, level):
  level = int_parameter(sample_level(level), WIDTH / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((WIDTH, HEIGHT),
                           Image.AFFINE, (1, 0, level, 0, 1, 0),
                           resample=Image.BILINEAR)


def translate_y(pil_img, level):
  level = int_parameter(sample_level(level), HEIGHT / 3)
  if np.random.random() > 0.5:
    level = -level
  return pil_img.transform((WIDTH, HEIGHT),
                           Image.AFFINE, (1, 0, 0, 0, 1, level),
                           resample=Image.BILINEAR)


# operation that overlaps with ImageNet-C's test set
def color(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Color(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def contrast(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Contrast(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def brightness(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Brightness(pil_img).enhance(level)


# operation that overlaps with ImageNet-C's test set
def sharpness(pil_img, level):
    level = float_parameter(sample_level(level), 1.8) + 0.1
    return ImageEnhance.Sharpness(pil_img).enhance(level)


augmentations = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y
]

augmentations_all = [
    autocontrast, equalize, posterize, rotate, solarize, shear_x, shear_y,
    translate_x, translate_y, color, contrast, brightness, sharpness
]

# Copyright 2019 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Reference implementation of AugMix's data augmentation method in numpy."""
import numpy as np
from PIL import Image

# CIFAR-10 constants
MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]


def normalize(image):
  """Normalize input image channel-wise to zero mean and unit variance."""
  image = image.transpose(2, 0, 1)  # Switch to channel-first
  mean, std = np.array(MEAN), np.array(STD)
  image = (image - mean[:, None, None]) / std[:, None, None]
  return image.transpose(1, 2, 0)


def apply_op(image, op, severity):
  image = np.clip(image * 255., 0, 255).astype(np.uint8)
  pil_img = Image.fromarray(image)  # Convert to PIL.Image
  pil_img = op(pil_img, severity)
  return np.asarray(pil_img) / 255.


def augment_and_mix(image, severity=3, width=3, depth=-1, alpha=1.):
  """Perform AugMix augmentations and compute mixture.
  Args:
    image: Raw input image as float32 np.ndarray of shape (h, w, c)
    severity: Severity of underlying augmentation operators (between 1 to 10).
    width: Width of augmentation chain
    depth: Depth of augmentation chain. -1 enables stochastic depth uniformly
      from [1, 3]
    alpha: Probability coefficient for Beta and Dirichlet distributions.
  Returns:
    mixed: Augmented and mixed image.
  """
  ws = np.float32(
      np.random.dirichlet([alpha] * width))
  m = np.float32(np.random.beta(alpha, alpha))

  mix = np.zeros_like(image)
  for i in range(width):
    image_aug = image.copy()
    depth = depth if depth > 0 else np.random.randint(1, 4)
    for _ in range(depth):
      op = np.random.choice(augmentations)
      image_aug = apply_op(image_aug, op, severity)
    # Preprocessing commutes since all coefficients are convex
#     mix += ws[i] * normalize(image_aug)
    mix += ws[i] * image_aug

#   mixed = (1 - m) * normalize(image) + m * mix
  mixed = (1 - m) * image + m * mix
  return mixed


from albumentations.core.transforms_interface import DualTransform
class GridMask(DualTransform):
    """GridMask augmentation for image classification and object detection.
    
    Author: Qishen Ha
    Email: haqishen@gmail.com
    2020/01/29

    Args:
        num_grid (int): number of grid in a row or column.
        fill_value (int, float, lisf of int, list of float): value for dropped pixels.
        rotate ((int, int) or int): range from which a random angle is picked. If rotate is a single int
            an angle is picked from (-rotate, rotate). Default: (-90, 90)
        mode (int):
            0 - cropout a quarter of the square of each grid (left top)
            1 - reserve a quarter of the square of each grid (left top)
            2 - cropout 2 quarter of the square of each grid (left top & right bottom)

    Targets:
        image, mask

    Image types:
        uint8, float32

    Reference:
    |  https://arxiv.org/abs/2001.04086
    |  https://github.com/akuxcw/GridMask
    """

    def __init__(self, num_grid=3, fill_value=0, rotate=0, mode=0, always_apply=False, p=0.5):
        super(GridMask, self).__init__(always_apply, p)
        if isinstance(num_grid, int):
            num_grid = (num_grid, num_grid)
        if isinstance(rotate, int):
            rotate = (-rotate, rotate)
        self.num_grid = num_grid
        self.fill_value = fill_value
        self.rotate = rotate
        self.mode = mode
        self.masks = None
        self.rand_h_max = []
        self.rand_w_max = []

    def init_masks(self, height, width):
        if self.masks is None:
            self.masks = []
            n_masks = self.num_grid[1] - self.num_grid[0] + 1
            for n, n_g in enumerate(range(self.num_grid[0], self.num_grid[1] + 1, 1)):
                grid_h = height / n_g
                grid_w = width / n_g
                this_mask = np.ones((int((n_g + 1) * grid_h), int((n_g + 1) * grid_w))).astype(np.uint8)
                for i in range(n_g + 1):
                    for j in range(n_g + 1):
                        this_mask[
                             int(i * grid_h) : int(i * grid_h + grid_h / 2),
                             int(j * grid_w) : int(j * grid_w + grid_w / 2)
                        ] = self.fill_value
                        if self.mode == 2:
                            this_mask[
                                 int(i * grid_h + grid_h / 2) : int(i * grid_h + grid_h),
                                 int(j * grid_w + grid_w / 2) : int(j * grid_w + grid_w)
                            ] = self.fill_value
                
                if self.mode == 1:
                    this_mask = 1 - this_mask

                self.masks.append(this_mask)
                self.rand_h_max.append(grid_h)
                self.rand_w_max.append(grid_w)

    def apply(self, image, mask, rand_h, rand_w, angle, **params):
        h, w = image.shape[:2]
        mask = F.rotate(mask, angle) if self.rotate[1] > 0 else mask
        mask = mask[:,:,np.newaxis] if image.ndim == 3 else mask
        image *= mask[rand_h:rand_h+h, rand_w:rand_w+w].astype(image.dtype)
        return image

    def get_params_dependent_on_targets(self, params):
        img = params['image']
        height, width = img.shape[:2]
        self.init_masks(height, width)

        mid = np.random.randint(len(self.masks))
        mask = self.masks[mid]
        rand_h = np.random.randint(self.rand_h_max[mid])
        rand_w = np.random.randint(self.rand_w_max[mid])
        angle = np.random.randint(self.rotate[0], self.rotate[1]) if self.rotate[1] > 0 else 0

        return {'mask': mask, 'rand_h': rand_h, 'rand_w': rand_w, 'angle': angle}

    @property
    def targets_as_params(self):
        return ['image']

    def get_transform_init_args_names(self):
        return ('num_grid', 'fill_value', 'rotate', 'mode')

class RandomAugMix(ImageOnlyTransform):

    def __init__(self, severity=3, width=3, depth=-1, alpha=1., always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.severity = severity
        self.width = width
        self.depth = depth
        self.alpha = alpha

    def apply(self, image, **params):
        image = augment_and_mix(
            image,
            self.severity,
            self.width,
            self.depth,
            self.alpha
        )
        return image

# https://github.com/DeepVoltaire/AutoAugment
from PIL import Image, ImageEnhance, ImageOps
import numpy as np
import random


class ImageNetPolicy(object):
    """ Randomly choose one of the best 24 Sub-policies on ImageNet.
        Example:
        >>> policy = ImageNetPolicy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     ImageNetPolicy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.4, "posterize", 8, 0.6, "rotate", 9, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor),
            SubPolicy(0.6, "posterize", 7, 0.6, "posterize", 6, fillcolor),
            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),

            SubPolicy(0.4, "equalize", 4, 0.8, "rotate", 8, fillcolor),
            SubPolicy(0.6, "solarize", 3, 0.6, "equalize", 7, fillcolor),
            SubPolicy(0.8, "posterize", 5, 1.0, "equalize", 2, fillcolor),
            SubPolicy(0.2, "rotate", 3, 0.6, "solarize", 8, fillcolor),
            SubPolicy(0.6, "equalize", 8, 0.4, "posterize", 6, fillcolor),

            SubPolicy(0.8, "rotate", 8, 0.4, "color", 0, fillcolor),
            SubPolicy(0.4, "rotate", 9, 0.6, "equalize", 2, fillcolor),
            SubPolicy(0.0, "equalize", 7, 0.8, "equalize", 8, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),

            SubPolicy(0.8, "rotate", 8, 1.0, "color", 2, fillcolor),
            SubPolicy(0.8, "color", 8, 0.8, "solarize", 7, fillcolor),
            SubPolicy(0.4, "sharpness", 7, 0.6, "invert", 8, fillcolor),
            SubPolicy(0.6, "shearX", 5, 1.0, "equalize", 9, fillcolor),
            SubPolicy(0.4, "color", 0, 0.6, "equalize", 3, fillcolor),

            SubPolicy(0.4, "equalize", 7, 0.2, "solarize", 4, fillcolor),
            SubPolicy(0.6, "solarize", 5, 0.6, "autocontrast", 5, fillcolor),
            SubPolicy(0.6, "invert", 4, 1.0, "equalize", 8, fillcolor),
            SubPolicy(0.6, "color", 4, 1.0, "contrast", 8, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.6, "equalize", 3, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment ImageNet Policy"


class CIFAR10Policy(object):
    """ Randomly choose one of the best 25 Sub-policies on CIFAR10.
        Example:
        >>> policy = CIFAR10Policy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     CIFAR10Policy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.1, "invert", 7, 0.2, "contrast", 6, fillcolor),
            SubPolicy(0.7, "rotate", 2, 0.3, "translateX", 9, fillcolor),
            SubPolicy(0.8, "sharpness", 1, 0.9, "sharpness", 3, fillcolor),
            SubPolicy(0.5, "shearY", 8, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.5, "autocontrast", 8, 0.9, "equalize", 2, fillcolor),

            SubPolicy(0.2, "shearY", 7, 0.3, "posterize", 7, fillcolor),
            SubPolicy(0.4, "color", 3, 0.6, "brightness", 7, fillcolor),
            SubPolicy(0.3, "sharpness", 9, 0.7, "brightness", 9, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.5, "equalize", 1, fillcolor),
            SubPolicy(0.6, "contrast", 7, 0.6, "sharpness", 5, fillcolor),

            SubPolicy(0.7, "color", 7, 0.5, "translateX", 8, fillcolor),
            SubPolicy(0.3, "equalize", 7, 0.4, "autocontrast", 8, fillcolor),
            SubPolicy(0.4, "translateY", 3, 0.2, "sharpness", 6, fillcolor),
            SubPolicy(0.9, "brightness", 6, 0.2, "color", 8, fillcolor),
            SubPolicy(0.5, "solarize", 2, 0.0, "invert", 3, fillcolor),

            SubPolicy(0.2, "equalize", 0, 0.6, "autocontrast", 0, fillcolor),
            SubPolicy(0.2, "equalize", 8, 0.6, "equalize", 4, fillcolor),
            SubPolicy(0.9, "color", 9, 0.6, "equalize", 6, fillcolor),
            SubPolicy(0.8, "autocontrast", 4, 0.2, "solarize", 8, fillcolor),
            SubPolicy(0.1, "brightness", 3, 0.7, "color", 0, fillcolor),

            SubPolicy(0.4, "solarize", 5, 0.9, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "translateY", 9, 0.7, "translateY", 9, fillcolor),
            SubPolicy(0.9, "autocontrast", 2, 0.8, "solarize", 3, fillcolor),
            SubPolicy(0.8, "equalize", 8, 0.1, "invert", 3, fillcolor),
            SubPolicy(0.7, "translateY", 9, 0.9, "autocontrast", 1, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment CIFAR10 Policy"


class SVHNPolicy(object):
    """ Randomly choose one of the best 25 Sub-policies on SVHN.
        Example:
        >>> policy = SVHNPolicy()
        >>> transformed = policy(image)
        Example as a PyTorch Transform:
        >>> transform=transforms.Compose([
        >>>     transforms.Resize(256),
        >>>     SVHNPolicy(),
        >>>     transforms.ToTensor()])
    """
    def __init__(self, fillcolor=(128, 128, 128)):
        self.policies = [
            SubPolicy(0.9, "shearX", 4, 0.2, "invert", 3, fillcolor),
            SubPolicy(0.9, "shearY", 8, 0.7, "invert", 5, fillcolor),
            SubPolicy(0.6, "equalize", 5, 0.6, "solarize", 6, fillcolor),
            SubPolicy(0.9, "invert", 3, 0.6, "equalize", 3, fillcolor),
            SubPolicy(0.6, "equalize", 1, 0.9, "rotate", 3, fillcolor),

            SubPolicy(0.9, "shearX", 4, 0.8, "autocontrast", 3, fillcolor),
            SubPolicy(0.9, "shearY", 8, 0.4, "invert", 5, fillcolor),
            SubPolicy(0.9, "shearY", 5, 0.2, "solarize", 6, fillcolor),
            SubPolicy(0.9, "invert", 6, 0.8, "autocontrast", 1, fillcolor),
            SubPolicy(0.6, "equalize", 3, 0.9, "rotate", 3, fillcolor),

            SubPolicy(0.9, "shearX", 4, 0.3, "solarize", 3, fillcolor),
            SubPolicy(0.8, "shearY", 8, 0.7, "invert", 4, fillcolor),
            SubPolicy(0.9, "equalize", 5, 0.6, "translateY", 6, fillcolor),
            SubPolicy(0.9, "invert", 4, 0.6, "equalize", 7, fillcolor),
            SubPolicy(0.3, "contrast", 3, 0.8, "rotate", 4, fillcolor),

            SubPolicy(0.8, "invert", 5, 0.0, "translateY", 2, fillcolor),
            SubPolicy(0.7, "shearY", 6, 0.4, "solarize", 8, fillcolor),
            SubPolicy(0.6, "invert", 4, 0.8, "rotate", 4, fillcolor),
            SubPolicy(0.3, "shearY", 7, 0.9, "translateX", 3, fillcolor),
            SubPolicy(0.1, "shearX", 6, 0.6, "invert", 5, fillcolor),

            SubPolicy(0.7, "solarize", 2, 0.6, "translateY", 7, fillcolor),
            SubPolicy(0.8, "shearY", 4, 0.8, "invert", 8, fillcolor),
            SubPolicy(0.7, "shearX", 9, 0.8, "translateY", 3, fillcolor),
            SubPolicy(0.8, "shearY", 5, 0.7, "autocontrast", 3, fillcolor),
            SubPolicy(0.7, "shearX", 2, 0.1, "invert", 5, fillcolor)
        ]


    def __call__(self, img):
        policy_idx = random.randint(0, len(self.policies) - 1)
        return self.policies[policy_idx](img)

    def __repr__(self):
        return "AutoAugment SVHN Policy"


class SubPolicy(object):
    def __init__(self, p1, operation1, magnitude_idx1, p2, operation2, magnitude_idx2, fillcolor=(128, 128, 128)):
        ranges = {
            "shearX": np.linspace(0, 0.3, 10),
            "shearY": np.linspace(0, 0.3, 10),
            "translateX": np.linspace(0, 150 / 331, 10),
            "translateY": np.linspace(0, 150 / 331, 10),
            "rotate": np.linspace(0, 30, 10),
            "color": np.linspace(0.0, 0.9, 10),
            "posterize": np.round(np.linspace(8, 4, 10), 0).astype(np.int),
            "solarize": np.linspace(256, 0, 10),
            "contrast": np.linspace(0.0, 0.9, 10),
            "sharpness": np.linspace(0.0, 0.9, 10),
            "brightness": np.linspace(0.0, 0.9, 10),
            "autocontrast": [0] * 10,
            "equalize": [0] * 10,
            "invert": [0] * 10
        }

        # from https://stackoverflow.com/questions/5252170/specify-image-filling-color-when-rotating-in-python-with-pil-and-setting-expand
        def rotate_with_fill(img, magnitude):
            rot = img.convert("RGBA").rotate(magnitude)
            return Image.composite(rot, Image.new("RGBA", rot.size, (128,) * 4), rot).convert(img.mode)

        func = {
            "shearX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, magnitude * random.choice([-1, 1]), 0, 0, 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "shearY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, magnitude * random.choice([-1, 1]), 1, 0),
                Image.BICUBIC, fillcolor=fillcolor),
            "translateX": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, magnitude * img.size[0] * random.choice([-1, 1]), 0, 1, 0),
                fillcolor=fillcolor),
            "translateY": lambda img, magnitude: img.transform(
                img.size, Image.AFFINE, (1, 0, 0, 0, 1, magnitude * img.size[1] * random.choice([-1, 1])),
                fillcolor=fillcolor),
            "rotate": lambda img, magnitude: rotate_with_fill(img, magnitude),
            "color": lambda img, magnitude: ImageEnhance.Color(img).enhance(1 + magnitude * random.choice([-1, 1])),
            "posterize": lambda img, magnitude: ImageOps.posterize(img, magnitude),
            "solarize": lambda img, magnitude: ImageOps.solarize(img, magnitude),
            "contrast": lambda img, magnitude: ImageEnhance.Contrast(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "sharpness": lambda img, magnitude: ImageEnhance.Sharpness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "brightness": lambda img, magnitude: ImageEnhance.Brightness(img).enhance(
                1 + magnitude * random.choice([-1, 1])),
            "autocontrast": lambda img, magnitude: ImageOps.autocontrast(img),
            "equalize": lambda img, magnitude: ImageOps.equalize(img),
            "invert": lambda img, magnitude: ImageOps.invert(img)
        }

        self.p1 = p1
        self.operation1 = func[operation1]
        self.magnitude1 = ranges[operation1][magnitude_idx1]
        self.p2 = p2
        self.operation2 = func[operation2]
        self.magnitude2 = ranges[operation2][magnitude_idx2]


    def __call__(self, img):
        if random.random() < self.p1: img = self.operation1(img, self.magnitude1)
        if random.random() < self.p2: img = self.operation2(img, self.magnitude2)
        return img
   
class Policy(ImageOnlyTransform):

    def __init__(self, always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.policy = ImageNetPolicy()
    def apply(self, img, **params):
        img = (img*255).astype(np.uint8)
        img = Image.fromarray(img)
        return np.array(self.policy(img))


import numpy as np
from PIL import Image

# CIFAR-10 constants
MEAN = [0.4914, 0.4822, 0.4465]
STD = [0.2023, 0.1994, 0.2010]


def normalize(image):
  """Normalize input image channel-wise to zero mean and unit variance."""
  image = image.transpose(2, 0, 1)  # Switch to channel-first
  mean, std = np.array(MEAN), np.array(STD)
  image = (image - mean[:, None, None]) / std[:, None, None]
  return image.transpose(1, 2, 0)


def apply_op(image, op, severity):
  image = np.clip(image * 255., 0, 255).astype(np.uint8)
  pil_img = Image.fromarray(image)  # Convert to PIL.Image
  pil_img = op(pil_img, severity)
  return np.asarray(pil_img) / 255.


def augment_and_mix(image, severity=3, width=3, depth=-1, alpha=1.):
  """Perform AugMix augmentations and compute mixture.
  Args:
    image: Raw input image as float32 np.ndarray of shape (h, w, c)
    severity: Severity of underlying augmentation operators (between 1 to 10).
    width: Width of augmentation chain
    depth: Depth of augmentation chain. -1 enables stochastic depth uniformly
      from [1, 3]
    alpha: Probability coefficient for Beta and Dirichlet distributions.
  Returns:
    mixed: Augmented and mixed image.
  """
  ws = np.float32(
      np.random.dirichlet([alpha] * width))
  m = np.float32(np.random.beta(alpha, alpha))

  mix = np.zeros_like(image)
  for i in range(width):
    image_aug = image.copy()
    depth = depth if depth > 0 else np.random.randint(1, 4)
    for _ in range(depth):
      op = np.random.choice(augmentations)
      image_aug = apply_op(image_aug, op, severity)
    # Preprocessing commutes since all coefficients are convex
#     mix += ws[i] * normalize(image_aug)
    mix += ws[i] * image_aug

#   mixed = (1 - m) * normalize(image) + m * mix
  mixed = (1 - m) * image + m * mix
  return mixed

class RandomAugMix(ImageOnlyTransform):

    def __init__(self, severity=3, width=3, depth=-1, alpha=1., always_apply=False, p=0.5):
        super().__init__(always_apply, p)
        self.severity = severity
        self.width = width
        self.depth = depth
        self.alpha = alpha

    def apply(self, image, **params):
        image = image.astype(np.float64)
        image = augment_and_mix(
            image,
            self.severity,
            self.width,
            self.depth,
            self.alpha
        )
        return image


from albumentations import Compose, ShiftScaleRotate, OneOf, Cutout
from PIL import Image




def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=CHANNELS)
    return img

def resize(img):
    return cv2.resize(img.numpy(), dsize=(WIDTH, HEIGHT))

def process_imgs(img_path, label):
    img = tf.io.read_file(img_path)
    img = decode_img(img)
    img = tf.image.convert_image_dtype(img, tf.float32)
    if SCALE != 1:
        shape = img.shape
        [img] = tf.py_function(resize, [img], [tf.float32])
        img.set_shape(shape)
    return img, label

def cache(ds, cache=None):
    if isinstance(cache, str):
        ds = ds.cache(cache)
    else:
        ds = ds.cache()
    return ds

def aug_method(p=1):
    return Compose([
        # ShiftScaleRotate(
        #     shift_limit=0.1, 
        #     scale_limit=0.1, 
        #     rotate_limit=45,
        #     border_mode = cv2.BORDER_CONSTANT,
        #     p=0),
        OneOf([
            # RandomAugMix(severity=1, width=1, alpha=1, p=0),
#         Cutout(num_holes=4, 
#                max_h_size=60, 
#                max_w_size=60, 
#                fill_value=0, 
#                p=0)]
        GridMask(num_grid=6, mode=0, p=1),
        GridMask(num_grid=12, mode=0, p=1),
        GridMask(num_grid=4, mode=0, p=1)])
    ],p=p)


def aug(img):
    f = aug_method()
    img = img.numpy()
    return f(image=img)["image"]

def img_aug(img, do_aug):
    shape = img.shape
    if do_aug==1:
        [img] = tf.py_function(aug, [img], [tf.float32])
    img.set_shape(shape)
    return img


def prepare_for_training(ds, shuffle=False):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if shuffle:
        ds = ds.shuffle(buffer_size=BATCH_SIZE*100)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

def mixup(image, label, p = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    imgs = [] 
    labs_root = []
    labs_vowel = []
    labs_cons = []
    for j in range(BATCH_SIZE):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=p, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,BATCH_SIZE),tf.int32)
        a = tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL
        lab1_root = label[0][j,]
        lab2_root = label[0][k,]
        labs_root.append((1-a)*lab1_root + a*lab2_root)
        
        lab1_vowel = label[1][j,]
        lab2_vowel = label[1][k,]
        labs_vowel.append((1-a)*lab1_vowel + a*lab2_vowel)
        
        lab1_cons = label[2][j,]
        lab2_cons = label[2][k,]
        labs_cons.append((1-a)*lab1_cons + a*lab2_cons)
    labs_root = tf.stack(labs_root)
    labs_vowel = tf.stack(labs_vowel)
    labs_cons = tf.stack(labs_cons)
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,HEIGHT,WIDTH,3))
    label2 = (labs_root, labs_vowel, labs_cons)
    return image2, label2   

def cutmix(image, label, p = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    imgs = [] 
    labs_root = []
    labs_vowel = []
    labs_cons = []
    for j in range(BATCH_SIZE):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=p, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,BATCH_SIZE),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,WIDTH),tf.int32)
        y = tf.cast( tf.random.uniform([],0,HEIGHT),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        width = tf.cast(WIDTH * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-width//2)
        yb = tf.math.minimum(HEIGHT,y+width//2)
        xa = tf.math.maximum(0,x-width//2)
        xb = tf.math.minimum(WIDTH,x+width//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:WIDTH,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:WIDTH,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(width*width/WIDTH/WIDTH,tf.float32)
        lab1_root = label[0][j,]
        lab2_root = label[0][k,]
        labs_root.append((1-a)*lab1_root + a*lab2_root)
        
        lab1_vowel = label[1][j,]
        lab2_vowel = label[1][k,]
        labs_vowel.append((1-a)*lab1_vowel + a*lab2_vowel)
        
        lab1_cons = label[2][j,]
        lab2_cons = label[2][k,]
        labs_cons.append((1-a)*lab1_cons + a*lab2_cons)
    labs_root = tf.stack(labs_root)
    labs_vowel = tf.stack(labs_vowel)
    labs_cons = tf.stack(labs_cons)
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(BATCH_SIZE,HEIGHT,WIDTH,3))
    label2 = (labs_root, labs_vowel, labs_cons)
    return image2,label2
    
def tf_img_aug(img, label):
    imgs = []
    for j in range(BATCH_SIZE):
        do_aug = tf.cast(tf.random.uniform([],0,1)<=IMG_AUG_PROB, tf.int32)
        img_transformed = img_aug(img[j], do_aug)
        imgs.append(img_transformed)
    img = tf.stack(imgs)
    if do_aug==0:
        CUTMIX_PROB = 0.66
        MIXUP_PROB = 0.66
        image2, label2 = cutmix(img, label, CUTMIX_PROB)
        image3, label3 = mixup(img, label, MIXUP_PROB)
        SWITCH = 0.5
        imgs = []
        labs_root = []
        labs_vowel = []
        labs_cons = []
        for j in range(BATCH_SIZE):
            P = tf.cast(tf.random.uniform([],0,1)<=SWITCH, tf.float32)
            imgs.append(P*image2[j,]+(1-P)*image3[j,])
            labs_root.append(P*label2[0][j,]+(1-P)*label3[0][j,])
            labs_vowel.append(P*label2[1][j,]+(1-P)*label3[1][j,])
            labs_cons.append(P*label2[2][j,]+(1-P)*label3[2][j,])
        img = tf.reshape(tf.stack(imgs),(BATCH_SIZE,HEIGHT,WIDTH,3))
        labs_root = tf.stack(labs_root)
        labs_vowel = tf.stack(labs_vowel)
        labs_cons = tf.stack(labs_cons)
        label= (labs_root, labs_vowel, labs_cons)
    return img,label
    

ds_train = ds_train.map(process_imgs, num_parallel_calls=AUTOTUNE)
ds_train = cache(ds_train, cache=(cache_path+"_train"))
ds_train = prepare_for_training(ds_train, shuffle=True)
ds_train = ds_train.map(tf_img_aug, num_parallel_calls=AUTOTUNE)
ds_train = ds_train.apply(tf.data.experimental.ignore_errors())

ds_val = ds_val.map(process_imgs, num_parallel_calls=AUTOTUNE)
ds_val = cache(ds_val, cache=(cache_path+"_val"))
ds_val = prepare_for_training(ds_val)

gc.collect()

if not model:
    gm_exp = tf.Variable(3.0, dtype = tf.float32)
    def generalized_mean_pool_2d(X):
        pool = (tf.reduce_mean(tf.abs(X**(gm_exp)), 
                        axis = [1, 2], 
                        keepdims = False) + 1.e-7)**(1./gm_exp)
        return pool
    inputs = layers.Input(shape=(HEIGHT, WIDTH, CHANNELS))
    base = se_resnext.SEResNextImageNet(depth=[3, 4, 23, 3], include_top=False, pooling=None, weights="imagenet", input_tensor=inputs)
    base.trainable = True

    # input = keras.Input(shape=(SIZE, SIZE, CHANNELS), name='original_img')

    x = layers.GlobalAveragePooling2D()(base.output)
    # lambda_layer = layers.Lambda(generalized_mean_pool_2d)
    # lambda_layer.trainable_weights.extend([gm_exp])
    # x = lambda_layer(base)
    output1 = layers.Dense(168, activation = 'softmax', name="root")(x)
    output2 = layers.Dense(11,activation = 'softmax', name="vowel")(x)
    output3 = layers.Dense(7, activation = 'softmax', name="consonant")(x)

    model = keras.Model(inputs, [output1, output2, output3])

else:
    model = keras.models.load_model(model)

model.compile(optimizer=keras.optimizers.Adam(),
            loss=keras.losses.CategoricalCrossentropy(from_logits=False),
            metrics=['categorical_accuracy']
            )
print(model.summary())

learning_rate_reduction_root = tf.keras.callbacks.ReduceLROnPlateau(monitor='root_categorical_accuracy', 
                                            patience=2, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_vowel = tf.keras.callbacks.ReduceLROnPlateau(monitor='vowel_categorical_accuracy', 
                                            patience=2, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)
learning_rate_reduction_consonant = tf.keras.callbacks.ReduceLROnPlateau(monitor='consonant_categorical_accuracy', 
                                            patience=2, 
                                            verbose=1,
                                            factor=0.5, 
                                            min_lr=0.00001)

csv_logger = tf.keras.callbacks.CSVLogger(model_histories_path)

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
    filepath=model_path,
    # Path where to save the model
    # The two parameters below mean that we will overwrite
    # the current checkpoint if and only if
    # the `val_loss` score has improved.
    save_best_only=False,
    monitor='val_loss',
    verbose=1),
    learning_rate_reduction_root,
    learning_rate_reduction_vowel,
    learning_rate_reduction_consonant,
    csv_logger
]
# config=model.get_config()
# (pd.DataFrame.from_dict(data=config)).to_csv(model_path+".csv", header=False)



Streaming output truncated to the last 5000 lines.
dense_12 (Dense)                (None, 1, 1, 32)     16384       reshape_6[0][0]                  
__________________________________________________________________________________________________
dense_13 (Dense)                (None, 1, 1, 512)    16384       dense_12[0][0]                   
__________________________________________________________________________________________________
multiply_6 (Multiply)           (None, 16, 16, 512)  0           batch_normalization_23[0][0]     
                                                                 dense_13[0][0]                   
__________________________________________________________________________________________________
add_6 (Add)                     (None, 16, 16, 512)  0           leaky_re_lu_18[0][0]             
                                                                 multiply_6[0][0]                 
__________________________________________________________

In [0]:
history = model.fit(
    ds_train,
    callbacks=callbacks,
    validation_data = ds_val,
    epochs=EPOCHS)

Train for 20084 steps, validate for 5021 steps
Epoch 1/60
20083/20084 [============================>.] - ETA: 0s - loss: 7.8367 - root_loss: 3.8580 - vowel_loss: 1.2331 - consonant_loss: 0.8396 - root_categorical_accuracy: 0.1585 - vowel_categorical_accuracy: 0.6420 - consonant_categorical_accuracy: 0.7292